Задача 3 про зевоту

In [1]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

In [5]:
def proportions_confint_diff_ind(a, b, n1, n2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    p1 = a/n1 #float(sum(sample1)) / len(sample1)
    p2 = b/n2 #float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n2 + p2 * (1 - p2)/ n2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    
    return (left_boundary, right_boundary)

In [6]:
print("confidence interval: [%f, %f]" % proportions_confint_diff_ind(10, 4, 34, 16))

confidence interval: [-0.263881, 0.305793]


In [7]:
a=10
b=4
n1=34
n2=16
p1 = a/n1
p2 = b/n2

In [8]:
p_=(p1*n1+p2*n2)/(n1+n2)
z=(p1-p2)/np.sqrt(p_*(1-p_)*(1/n1+1/n2))

In [9]:
z

0.32410186177608225

In [11]:
1-2*stats.norm.cdf(-z)

0.2541390825495293

Задача 4 про банкноты

In [13]:
df=pd.read_csv('banknotes.txt', sep='\t')
df.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [14]:
target=df['real']

In [19]:
from sklearn.model_selection import train_test_split

In [22]:
x_train, x_test, y_train, y_test = train_test_split(df, target, test_size=0.25, random_state=1)

In [23]:
data1_train=x_train.drop(['X4', 'X5', 'X6', 'real'], axis=1)
data1_test=x_test.drop(['X4', 'X5', 'X6', 'real'], axis=1)
data1_train.head()

,X1,X2,X3
98,215.1,130.0,129.8
123,215.1,130.7,130.4
119,214.4,130.2,129.9
53,215.4,130.2,130.2
33,215.6,130.4,130.1


In [24]:
data2_train=x_train.drop(['X1', 'X2', 'X3', 'real'], axis=1)
data2_test=x_test.drop(['X1', 'X2', 'X3', 'real'], axis=1)
data2_train.head()

,X4,X5,X6
98,9.1,10.2,141.5
123,10.5,11.2,139.7
119,10.1,12.0,139.2
53,7.6,10.9,141.6
33,8.4,10.3,141.0


In [26]:
from sklearn.linear_model import LogisticRegression

In [27]:
l1=LogisticRegression()
l2=LogisticRegression()

In [28]:
l1.fit(data1_train, y_train)
l2.fit(data2_train, y_train)

C:\Program Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Program Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [62]:
results=[l1.predict(data1_test), l2.predict(data2_test), np.array(y_test)]
pr1=l1.predict(data1_test)
pr2=l2.predict(data2_test)

In [61]:
pd.DataFrame(results).transpose()

,0,1,2
0,1,1,1
1,1,1,1
2,1,1,1
3,0,0,0
4,0,0,0
5,0,0,0
6,1,1,1
7,1,1,1
8,1,1,1
9,1,0,0


In [67]:
massiv1=np.abs(pr1-np.array(y_test))

In [68]:
massiv2=np.abs(pr2-np.array(y_test))

In [69]:
def proportions_confint_diff_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [70]:
print("confidence interval: [%f, %f]" % proportions_confint_diff_rel(massiv1, massiv2))

confidence interval: [0.059945, 0.300055]


In [71]:
f=10
g=1
n=50
z=(f-g)/np.sqrt(f+g-(f-g)**2/n)

In [72]:
z

2.9386041680175268

In [73]:
2*stats.norm.cdf(-z)

0.0032969384555543023

Задача о GMAT

In [77]:
mu=525
sigma=100
n=100
x_=541.5

In [78]:
z=(x_-mu)/sigma*np.sqrt(n)

In [79]:
stats.norm.cdf(-z)

0.049471468033648075